In [1]:
from IPython.display import Audio

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm import tqdm

from music.wav import read_wav, write_wav

tf.__version__

2021-11-17 23:13:39.781299: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-17 23:13:39.781323: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


'2.6.0'

In [2]:
dataset_path = 'dataset/genres_original/'

genres = [folder for folder in os.listdir(dataset_path)]

tracks = {
    genre: list(map(
        lambda filename: os.path.join(dataset_path, genre, filename),
        os.listdir(os.path.join(dataset_path, genre))
    ))
    for genre in genres
}

genres

['metal',
 'blues',
 'pop',
 'hiphop',
 'classical',
 'reggae',
 'country',
 'disco',
 'jazz',
 'rock']

In [2]:
music_file = tracks[genres[-1]][0]

track = read_wav(music_file)

Audio(music_file)
music_file

KeyboardInterrupt: 

In [ ]:
track

In [ ]:
dataset = dict()
dataset['disco'] = tracks['disco']
dataset['rock'] = tracks['rock']

dataset = {
    genre: [read_wav(filename) for filename in files]
    for genre, files in dataset.items()
}

dataset['rock'][0]

In [ ]:
# All tracks have same sample rate
np.unique(
    [t.sample_rate
     for _, v in dataset.items()
     for t in v]
)

SAMPLE_RATE = track.sample_rate
SAMPLE_RATE

In [ ]:
def flatten(list_of_lists):
    return [elem for lst in tqdm(list_of_lists) for elem in lst]


train_ratio = 0.8
test_ratio = 1 - train_ratio

X_rock = flatten([track.data for track in dataset['rock'][:10]])
X_disco = flatten([track.data for track in dataset['disco'][:10]])

y_rock, X_rock = X_rock[1:], X_rock[:-1]
y_disco, X_disco = X_disco[1:], X_disco[:-1]

X = X_rock + X_disco
y = y_rock + y_disco

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9)

len(X_train), len(X_test)

In [ ]:
lstm = Sequential([
    layers.Input((1, 3)),
    layers.LSTM(16),
    layers.Dense(16, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1)
])

lstm.summary()

lstm.compile(
    loss='mean_squared_error',
    optimizer='adam',
    metrics=['mean_squared_error'],
)

In [ ]:
history = lstm.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=10,
    callbacks=None,
    validation_split=0.2,
    verbose=2
)